In [74]:
def qc_genotype(genotype):
    """
    Check if the genotype is in the correct format.
    """
    if not isinstance(genotype, str):
        return False, "Genotype must be a string"
    
    # Check if the genotype is in the correct format xchromosome; chromosome2; chromosome3; chromosome4
    if not genotype.count(";") == 3:
        return False, "Genotype must be in the format xchromosome; chromosome2; chromosome3; chromosome4"
    
    # clean the genotype
    genotype = genotype.split(";")
    genotype = [x.strip() for x in genotype]
    # make sure each chromosome is in the correct format (atmost 1 '/' symbol)
    if any([x.count("/") > 1 for x in genotype]):
        return False, "Chromosomes must be in the format chromosomeA/chromosomeB (heterozygous) or chromosomeBoth (homozygous)"
    
    chrs = []
    for chr in genotype:
        # check if the chromosome is in the correct format
        if chr.count("/") == 1:
            # arrange the chromosome in the alphabetical order
            chr = chr.split("/")
            chr.sort()
            chr = "/".join(chr)
        chrs.append(chr)
    
    genotype = "; ".join(chrs)

    return True, genotype
    
def get_genetic_components(genotype, sex):
    """
    Get the chromosomes from the genotype.
    """
    assert sex in ["male","female"], "Sex must be either male or female"
    chromosomes = genotype.split(";")
    chromosomes = [x.strip() for x in chromosomes]
    # for each chromosome, get the alleles
    components = []
    # handle the X chromosome
    if chromosomes[0] == "":
        components.append(["+", "+"] if sex == "female" else ["+","0"])
    else:
        assert chromosomes[0].count("/") == 0, "X chromosome should not have a '/'"
        components.append([chromosomes[0], chromosomes[0]] if sex == "female" else [chromosomes[0], "0"])
    for chr in chromosomes[1:]:
        if chr.count("/") == 1:
            alleles = chr.split("/")
            alleles.sort()
            # replace "" with "+"
            alleles = [allele if allele != "" else "+" for allele in alleles]
            components.append(alleles)
        else:
            components.append([chr if chr != "" else "+", chr if chr != "" else "+"])
    return components

# Example usage
output = get_genetic_components("w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2;","male") 
print(output)

def get_genotype_from_components(components):
    """
    Convert genetic components back into a genotype string, inferring sex from the X chromosome.
    If both alleles are '+', the field will be left empty (for homozygous '+' cases).
    """
    # Prepare the list to store chromosomes
    genotype = []

    sex = "female"
    
    # Handle the X chromosome (infer sex from the first chromosome component)
    x_chromosome = components[0]
    if "0" in x_chromosome:
        # Male case (X0), one of the X alleles is "0"
        # append the other allele
        genotype.append(x_chromosome[0] if x_chromosome[0] != "0" else x_chromosome[1])
        sex = "male"

    else:
        # Female case (XX or heterozygous X chromosomes)
        if x_chromosome[0] == x_chromosome[1]:
            # Homozygous X chromosome case
            genotype.append(x_chromosome[0] if x_chromosome[0] != "+" else "")
        else:
            # Heterozygous X chromosomes case
            x_chromosome.sort()  # Sort to maintain consistency
            genotype.append("/".join(x_chromosome))
    
    # Handle the autosomal chromosomes
    for chr_components in components[1:]:
        if chr_components[0] == chr_components[1]:
            # Homozygous case (e.g., both alleles are the same)
            # Leave empty if both are '+'
            genotype.append("" if chr_components[0] == "+" else chr_components[0])
        else:
            # Heterozygous case (e.g., different alleles)
            chr_components.sort()  # Ensure the alleles are sorted alphabetically
            genotype.append("/".join(chr_components))
    
    # Join the chromosomes with "; " and return the genotype string
    return qc_genotype("; ".join(genotype))[1], sex

# Example usage
components = [['w[1118]', 'w[1]'],
              ['+', 'CyO'],
              ['P{20XUAS-CsChrimson-mVenus trafficked}attP2', 'TM2'],
              ['+', '+']]

genotype = get_genotype_from_components(output)
print(genotype)


[['w[1118]', '0'], ['+', 'CyO'], ['P{20XUAS-CsChrimson-mVenus trafficked}attP2', 'TM2'], ['+', '+']]
('w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ', 'male')


In [78]:
# create a function to cross two genotypes
from itertools import product
import numpy as np

def sort_components_alphabetically(t):
    """
    Sort the alleles in each chromosome pair alphabetically.
    """
    return [list(sorted(x)) for x in t]

def cross_genotypes(male_genotype,female_genotype):
    """
    Cross two genotypes.
    """
    male_components = get_genetic_components(qc_genotype(male_genotype)[1],"male")
    female_components = get_genetic_components(qc_genotype(female_genotype)[1],"female")
    # get all possible combinations of chromosomes for each chromosome pair
    combinations = [sort_components_alphabetically(list(product(m,f))) for m,f in zip(male_components,female_components)]
    # get all possible combinations of chromosomes
    combinations = list(product(*combinations))
    # convert the combinations into genotypes
    genotypes = ["|".join(get_genotype_from_components(list(comb))) for comb in combinations]
    # get the unique genotypes along with probabilities
    genotypes, counts = np.unique(genotypes, return_counts=True)
    probabilities = counts/np.sum(counts)
    # sort the genotypes by probability
    indices = np.argsort(probabilities)[::-1]
    genotypes = genotypes[indices]
    probabilities = probabilities[indices]
    # convert back to [genotype, sex, probability] format
    combinations = [[genotype.split("|")[0],genotype.split("|")[1],probability] for genotype,probability in zip(genotypes,probabilities)]

    return combinations

test = cross_genotypes("w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2;","w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2;")
    

In [79]:
test

[['w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ',
  'female',
  0.125],
 ['w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ',
  'male',
  0.125],
 ['w[1118]; ; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ',
  'male',
  0.0625],
 ['w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2; ',
  'female',
  0.0625],
 ['w[1118]; CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ',
  'male',
  0.0625],
 ['w[1118]; CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ',
  'female',
  0.0625],
 ['w[1118]; +/CyO; P{20XUAS-CsChrimson-mVenus trafficked}attP2; ',
  'male',
  0.0625],
 ['w[1118]; +/CyO; TM2; ', 'female', 0.0625],
 ['w[1118]; +/CyO; TM2; ', 'male', 0.0625],
 ['w[1118]; ; P{20XUAS-CsChrimson-mVenus trafficked}attP2/TM2; ',
  'female',
  0.0625],
 ['w[1118]; CyO; TM2; ', 'male', 0.03125],
 ['w[1118]; CyO; TM2; ', 'female', 0.03125],
 ['w[1118]; ; P{20XUAS-CsChrimson-mVenus trafficked}attP2; ', 'male', 0.03125],
 ['w[1118]; ; TM2; ', 

In [1]:
# internal imports
from flymanager.utils.mongo import *
from flymanager.utils.labels import *
from flymanager.utils.scanner import *
from flymanager.utils.utils import *
from flymanager.utils.converter import *
from flymanager.utils.genetics import *

# setup dotenv
from dotenv import load_dotenv
load_dotenv()

from datetime import datetime, datetime.timedelta

# setup the mongo db
client = create_mongo_client()
db = get_database(client)

In [2]:
# check if each one has a vial name and a date in the correct format: (V#, YYYY-MM-DD HH:MM)
def clean_log_entry(entry):
    """ 
    Clean the log entry to get the vial and the date.
    """
    if entry.count(",") != 1:
        vial = None
        try:
            date = datetime.strptime(entry.strip(), "%Y-%m-%d %H:%M")
        except:
            date = datetime.strptime(entry.strip(), "%Y-%m-%d %H:%M:%S")
        return vial, date
    else:
        vial = entry.split(", ")[0].strip()
        try:
            date = datetime.strptime(entry.split(",")[1].strip(), "%Y-%m-%d %H:%M")
        except:
            date = datetime.strptime(entry.split(",")[1].strip(), "%Y-%m-%d %H:%M:%S")
        return vial, date

In [3]:



def update_stock_vials(stock, username, db):
    """
    Refresh the stock vials based on the flip log.

    Parameters:
    stock : dict
        The stock dictionary.
    username : str
        The username of the user.
    db : pymongo.database.Database
        The database object.

    Returns:
    bool
        True if the stock was updated successfully, False otherwise.
    """
    uid = stock["UniqueID"]

    # check if the stock doesnt have the key "CurrentlyAliveVials"
    if "CurrentlyAliveVials" not in stock:
        assert all([x in stock for x in ["FlipLog","VialLifetime","FlipFrequency","DevelopmentalTime"]]), "Stock must have the keys FlipLog, VialLifetime, and FlipFrequency"
        
        # get the stock details
        flipLog = stock["FlipLog"]
        vialLifetime = float(stock["VialLifetime"])
        flipFrequency = float(stock["FlipFrequency"])
        developmentalTime = float(stock["DevelopmentalTime"])
        # split the flip log into a list by ";"
        flipLog = flipLog.split(";")
        _, flip_dates = zip(*[clean_log_entry(flip) for flip in flipLog])

        # reverse the order of the vials and dates
        flip_dates = flip_dates[::-1]
            
        vials = ["V{}".format(i) for i in range(1,len(flip_dates)+1)]
        next_flip_dates = [date+datetime.timedelta(days=flipFrequency) for date in flip_dates]
        next_eclosion_dates = [date+datetime.timedelta(days=developmentalTime) for date in flip_dates]

        # recreate a new flip log
        flipLog = [f"{vial}, {date.strftime('%Y-%m-%d %H:%M')}" for vial, date in zip(vials[::-1],flip_dates[::-1])]
        flipLog = "; ".join(flipLog)

        # remove HH:MM:SS from the dates
        flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in flip_dates]
        next_flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_flip_dates]
        next_eclosion_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_eclosion_dates]

        # determine the dead vials
        currently_alive_vials = vials.copy()
        to_delete = []
        for vial, flip_date, next_flip_date in zip(vials,flip_dates,next_flip_dates):
            death_date = flip_date + datetime.timedelta(days=vialLifetime)
            # if death date has passed and there is atleast one flip AFTER the scheduled death date
            if datetime.datetime.now() > death_date and any([date >= next_flip_date for date in flip_dates]):
                # get the index of the vial
                index = currently_alive_vials.index(vial)
                to_delete.append(index)
                print(f"Vial {vial} for stock {uid} has died")
        # remove the dead vials
        currently_alive_vials = [vial for i,vial in enumerate(currently_alive_vials) if i not in to_delete] 
        next_flip_dates = [date for i,date in enumerate(next_flip_dates) if i not in to_delete]
        next_eclosion_dates = [date for i,date in enumerate(next_eclosion_dates) if i not in to_delete]

        # define the currently alive vials
        currently_alive_vials = ", ".join(currently_alive_vials)
        # define the next flip dates
        next_flip_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_flip_dates])
        # define the next eclosion dates
        next_eclosion_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_eclosion_dates])
        # update the stock
        update_properties = {
            "FlipLog":flipLog,
            "CurrentlyAliveVials":currently_alive_vials,
            "NextFlipDates":next_flip_dates,
            "NextEclosionDates":next_eclosion_dates
        }
        # edit the stock
        success = edit_stock(username, uid, db, update_properties, log_activity=False)
        
    else:
        # get all flip dates
        flipLog = stock["FlipLog"]
        flipLog = flipLog.split(";")
        date_map = {}
        flip_dates = []

        for flip in flipLog:
            vial, date = clean_log_entry(flip)
            date_map[vial] = date
            flip_dates.append(date)

        # get the currently alive vials
        currently_alive_vials = stock["CurrentlyAliveVials"].split(", ")
        # keep only the alive vials
        dates = [date_map[vial] for vial in currently_alive_vials]
        # get the next flip dates
        flipFrequency = float(stock["FlipFrequency"])
        next_flip_dates = [date+datetime.timedelta(days=flipFrequency) for date in dates]
        # get the next eclosion dates
        developmentalTime = float(stock["DevelopmentalTime"])
        next_eclosion_dates = [date+datetime.timedelta(days=developmentalTime) for date in dates]

        # remove HH:MM:SS from the dates
        flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in flip_dates]
        next_flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_flip_dates]
        next_eclosion_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_eclosion_dates]

        # determine the dead vials
        vialLifetime = float(stock["VialLifetime"])
        to_delete = []
        for vial, flip_date, next_flip_date in zip(currently_alive_vials,dates,next_flip_dates):
            death_date = flip_date + datetime.timedelta(days=vialLifetime)
            # if death date has passed and there is atleast one flip AFTER the scheduled death date
            if datetime.datetime.now() > death_date and any([date >= next_flip_date for date in flip_dates]):
                # get the index of the vial
                index = currently_alive_vials.index(vial)
                to_delete.append(index)
                print(f"Vial {vial} for stock {uid} has died")
        
        # remove the dead vials
        currently_alive_vials = [vial for i,vial in enumerate(currently_alive_vials) if i not in to_delete]
        next_flip_dates = [date for i,date in enumerate(next_flip_dates) if i not in to_delete]
        next_eclosion_dates = [date for i,date in enumerate(next_eclosion_dates) if i not in to_delete]

        # define the currently alive vials
        currently_alive_vials = ", ".join(currently_alive_vials)
        # define the next flip dates
        next_flip_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_flip_dates])
        # define the next eclosion dates
        next_eclosion_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_eclosion_dates])
        # update the stock
        update_properties = {
            "CurrentlyAliveVials":currently_alive_vials,
            "NextFlipDates":next_flip_dates,
            "NextEclosionDates":next_eclosion_dates
        }
        # edit the stock
        success = edit_stock(username, uid, db, update_properties, log_activity=False)
    return success
    

In [20]:
stocks = get_user_stocks("rmohanta", db)
for stock in stocks:
    update_stock_vials(stock, db)

In [4]:
def update_cross_vials(cross, username, db):
    """
    Refresh the cross vials based on the flip log.

    Parameters:
    cross : dict
        The cross dictionary.   
    username : str
        The username of the user.
    db : pymongo.database.Database
        The database object.

    Returns:
    bool
        True if the cross was updated successfully, False otherwise.
    """
    uid = cross["UniqueID"]

    # check if the cross doesnt have the key "CurrentlyAliveVials"
    if "CurrentlyAliveVials" not in cross:
        assert all([x in cross for x in ["FlipLog","VialLifetime","FlipFrequency","DevelopmentalTime"]]), "Stock must have the keys FlipLog, VialLifetime, and FlipFrequency"
        
        # get the cross details
        flipLog = cross["FlipLog"]
        vialLifetime = float(cross["VialLifetime"])
        flipFrequency = float(cross["FlipFrequency"])
        developmentalTime = float(cross["DevelopmentalTime"])
        # split the flip log into a list by ";"
        flipLog = flipLog.split(";")
        _, flip_dates = zip(*[clean_log_entry(flip) for flip in flipLog])

        # reverse the order of the vials and dates
        flip_dates = flip_dates[::-1]
            
        vials = ["V{}".format(i) for i in range(1,len(flip_dates)+1)]
        next_flip_dates = [date+datetime.timedelta(days=flipFrequency) for date in flip_dates]
        next_eclosion_dates = [date+datetime.timedelta(days=developmentalTime) for date in flip_dates]

        # recreate a new flip log
        flipLog = [f"{vial}, {date.strftime('%Y-%m-%d %H:%M')}" for vial, date in zip(vials[::-1],flip_dates[::-1])]
        flipLog = "; ".join(flipLog)

        # remove HH:MM:SS from the dates
        flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in flip_dates]
        next_flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_flip_dates]
        next_eclosion_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_eclosion_dates]

        # determine the dead vials
        currently_alive_vials = vials.copy()
        to_delete = []
        for vial, flip_date, next_flip_date in zip(vials,flip_dates,next_flip_dates):
            death_date = flip_date + datetime.timedelta(days=vialLifetime)
            # if death date has passed and there is atleast one flip AFTER the scheduled death date
            if datetime.datetime.now() > death_date and any([date >= next_flip_date for date in flip_dates]):
                # get the index of the vial
                index = currently_alive_vials.index(vial)
                to_delete.append(index)
        # remove the dead vials
        currently_alive_vials = [vial for i,vial in enumerate(currently_alive_vials) if i not in to_delete] 
        next_flip_dates = [date for i,date in enumerate(next_flip_dates) if i not in to_delete]
        next_eclosion_dates = [date for i,date in enumerate(next_eclosion_dates) if i not in to_delete]

        # define the currently alive vials
        currently_alive_vials = ", ".join(currently_alive_vials)
        # define the next flip dates
        next_flip_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_flip_dates])
        # define the next eclosion dates
        next_eclosion_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_eclosion_dates])
        # update the cross
        update_properties = {
            "FlipLog":flipLog,
            "CurrentlyAliveVials":currently_alive_vials,
            "NextFlipDates":next_flip_dates,
            "NextEclosionDates":next_eclosion_dates
        }
        print(update_properties)
        # edit the cross
        success = edit_cross(username, uid, db, update_properties, log_activity=False)
        
    else:
        # get all flip dates
        flipLog = cross["FlipLog"]
        flipLog = flipLog.split(";")
        date_map = {}
        flip_dates = []
        for flip in flipLog:
            vial, date = clean_log_entry(flip)
            date_map[vial] = date
            flip_dates.append(date)

        # get the currently alive vials
        currently_alive_vials = cross["CurrentlyAliveVials"].split(", ")
        # keep only the alive vials
        dates = [date_map[vial] for vial in currently_alive_vials]
        # get the next flip dates
        flipFrequency = float(cross["FlipFrequency"])
        next_flip_dates = [date+datetime.timedelta(days=flipFrequency) for date in dates]
        # get the next eclosion dates
        developmentalTime = float(cross["DevelopmentalTime"])
        next_eclosion_dates = [date+datetime.timedelta(days=developmentalTime) for date in dates]

        # remove HH:MM:SS from the dates
        flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in flip_dates]
        next_flip_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_flip_dates]
        next_eclosion_dates = [date.replace(hour=0, minute=0, second=0, microsecond=0) for date in next_eclosion_dates]

        # determine the dead vials
        vialLifetime = float(cross["VialLifetime"])
        to_delete = []
        for vial, flip_date, next_flip_date in zip(currently_alive_vials,dates,next_flip_dates):
            death_date = flip_date + datetime.timedelta(days=vialLifetime)
            # if death date has passed and there is atleast one flip AFTER the scheduled death date
            if datetime.datetime.now() > death_date and any([date >= next_flip_date for date in flip_dates]):
                # get the index of the vial
                index = currently_alive_vials.index(vial)
                to_delete.append(index)
                
        # remove the dead vials
        currently_alive_vials = [vial for i,vial in enumerate(currently_alive_vials) if i not in to_delete]
        next_flip_dates = [date for i,date in enumerate(next_flip_dates) if i not in to_delete]
        next_eclosion_dates = [date for i,date in enumerate(next_eclosion_dates) if i not in to_delete]

        # define the currently alive vials
        currently_alive_vials = ", ".join(currently_alive_vials)
        # define the next flip dates
        next_flip_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_flip_dates])
        # define the next eclosion dates
        next_eclosion_dates = ", ".join([date.strftime('%Y-%m-%d') for date in next_eclosion_dates])
        # update the cross
        update_properties = {
            "CurrentlyAliveVials":currently_alive_vials,
            "NextFlipDates":next_flip_dates,
            "NextEclosionDates":next_eclosion_dates
        }
        print(update_properties)
        # edit the cross
        success = edit_cross(username, uid, db, update_properties, log_activity=False)
    return success
    

In [5]:
crosses = get_user_crosses("rmohanta", db)
for cross in crosses:
    update_cross_vials(cross, db)

Vial V1 for cross cff18930e8 has died
Vial V2 for cross cff18930e8 has died
Vial V3 for cross cff18930e8 has died
{'FlipLog': 'V4, 2024-09-13 17:26; V3, 2024-09-11 16:06; V2, 2024-09-09 17:05; V1, 2024-09-06 10:42', 'CurrentlyAliveVials': 'V4', 'NextFlipDates': '2024-09-15 17:26', 'NextEclosionDates': '2024-09-23 17:26'}
Vial V1 for cross 54c90e2400 has died
Vial V2 for cross 54c90e2400 has died
Vial V3 for cross 54c90e2400 has died
{'FlipLog': 'V4, 2024-09-20 18:30; V3, 2024-09-13 17:28; V2, 2024-09-09 17:05; V1, 2024-09-06 10:55', 'CurrentlyAliveVials': 'V4', 'NextFlipDates': '2024-09-22 18:30', 'NextEclosionDates': '2024-09-30 18:30'}
Vial V1 for cross 1a44e9d9f0 has died
Vial V2 for cross 1a44e9d9f0 has died
Vial V3 for cross 1a44e9d9f0 has died
{'FlipLog': 'V4, 2024-09-13 17:31; V3, 2024-09-11 16:11; V2, 2024-09-09 17:07; V1, 2024-09-06 10:56', 'CurrentlyAliveVials': 'V4', 'NextFlipDates': '2024-09-15 17:31', 'NextEclosionDates': '2024-09-23 17:31'}
Vial V1 for cross cb39d48f42 ha

In [10]:
stocks = get_user_stocks("rmohanta", db)

In [11]:
stocks[0]
import datetime

def get_flip_in(item):
    """
    Get the flip in for a stock or cross.
    """
    vals = []
    for val in item["NextFlipDates"].split(", "):
        now = datetime.datetime.now()
        vals.append(int((datetime.datetime.strptime(val, "%Y-%m-%d") - now).days))
    return ", ".join([str(val) for val in vals]) + " days"

def get_eclosion_in(item):
    """
    Get the eclosion in for a stock or cross.
    """
    vals = []
    for val in item["NextEclosionDates"].split(", "):
        now = datetime.datetime.now()
        vals.append(int((datetime.datetime.strptime(val, "%Y-%m-%d") - now).days))
    return ", ".join([str(val) for val in vals]) + " days"
    

In [13]:
stocks[0]

{'_id': ObjectId('66f587f7ca81b88d47b2b38d'),
 'UniqueID': 'c95939d9f8',
 'SourceID': 'BDSC_64349',
 'Genotype': 'w[CS]; ; ; ',
 'Name': 'Canton-S',
 'AltReference': '',
 'Type': 'WT',
 'SeriesID': '1',
 'ReplicateID': 'a',
 'TrayID': 'A',
 'TrayPosition': '1',
 'Status': 'Healthy',
 'FoodType': 'Wurtzburg',
 'Provenance': 'cmorton@RU',
 'Comments': '',
 'CreationDate': '2024-08-12 14:30:15',
 'LastFlipDate': '2024-09-20 15:25',
 'VialLifetime': '14',
 'DevelopmentalTime': '10',
 'FlipFrequency': '7',
 'FlipLog': 'V11, 2024-09-20 15:25; V10, 2024-09-13 17:09; V9, 2024-09-09 17:14; V8, 2024-09-06 17:16; V7, 2024-09-02 12:48; V6, 2024-08-23 13:56; V5, 2024-08-21 15:49; V4, 2024-08-19 16:44; V3, 2024-08-16 16:14; V2, 2024-08-14 17:51; V1, 2024-08-12 18:05',
 'DataModifiedDate': '',
 'ModificationLog': '',
 'User': 'rmohanta',
 'CurrentlyAliveVials': 'V10, V11',
 'NextEclosionDates': '2024-09-23, 2024-09-30',
 'NextFlipDates': '2024-09-20, 2024-09-27'}